In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## ***CONFIGURAÇÕES INICIAIS***

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import seaborn as sns

In [ ]:
# definir quantidade de colunas e linhas da tab
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 10000)

# omitir os warnings
np.warnings.filterwarnings('ignore')

# mudar formato de número no describe()
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## ***CARREGAR OS DADOS***

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
valid = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, valid.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(valid)

print('Linhas: ' + str(df_all.shape[0]))
print('Colunas: ' + str(df_all.shape[1]))

funções

In [ ]:
def histograma(var,linf,sup,inter=1):
    plt.figure(figsize=(15,5))

    plt.subplot(1, 2, 1)
    base_evasao_0 = base_evasao[base_evasao['FL_EVADIDO'] == 0]
    base_evasao_1 = base_evasao[base_evasao['FL_EVADIDO'] == 1]
    plt.hist(base_evasao_0[var],bins=list(range(linf, sup, inter)),label='Não evadido')
    plt.hist(base_evasao_1[var],bins=list(range(linf, sup, inter)),label='Evadido')
    plt.legend()
    plt.title(var)

    plt.subplot(1, 2, 2)
    sns.boxplot(x='FL_EVADIDO',y=var, data=base_evasao)
    plt.title(var)

In [ ]:
def exploratoria(var):
    print(df_all[var].describe())
    
    plt.figure(figsize=(10,5))

    plt.subplot(1, 2, 1)
    sns.boxplot(x='Target',y=var,data=df_all)
    plt.title(var)

    plt.subplot(1, 2, 2)
    df_all[var].hist(bins=50)
    plt.title(var)

In [ ]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10, duplicates='drop')
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    print(len(kstable))
    kstable.index = range(1,len(kstable)+1)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

## ***INFORMAÇÃO DOS DADOS***

In [ ]:
print(df_all.columns)

#### Quantidade de nulos

In [ ]:
# Visualizar os campos que possuem valores nulos
df_null = df_all.isnull().sum()
df_null[df_all.isnull().sum() != 0]

#### Tratamento dos nulos

var: v2a1

In [ ]:
print(df_all['v2a1'].describe())
df_all['v2a1'].fillna(df_all['v2a1'].mean(),inplace=True)

In [ ]:
df_all['Target'] == 1

var: v18q1

In [ ]:
print(df_all['v18q1'].mode()[0])
df_all['v18q1'].value_counts()

In [ ]:
df_all['v18q1'].fillna(df_all['v18q1'].mode()[0],inplace=True)

var: rez_esc

In [ ]:
print(df_all['rez_esc'].mode()[0])
df_all['rez_esc'].value_counts()

In [ ]:
df_all['rez_esc'].fillna(df_all['rez_esc'].mode()[0],inplace=True)

var: meaneduc

In [ ]:
print(df_all['meaneduc'].mean())
df_all['meaneduc'].describe()

In [ ]:
df_all['meaneduc'].fillna(df_all['meaneduc'].mean(),inplace=True)

var: SQBmeaned

In [ ]:
print(df_all['SQBmeaned'].mean())
df_all['SQBmeaned'].describe()

In [ ]:
df_all['SQBmeaned'].fillna(df_all['SQBmeaned'].mean(),inplace=True)

## ***ANÁLISE EXPLORATÓRIA***

In [ ]:
df_all['Target'].value_counts(), df_all['Target'].value_counts(normalize=True)

v2a1 (Monthly rent payment)

In [ ]:
exploratoria('v2a1')

agesq (Age squared)

In [ ]:
exploratoria('agesq')

SQBage (age squared)

In [ ]:
exploratoria('SQBage')

SQBedjefe (edjefe squared, edjefe: years of education of male head of household)

In [ ]:
exploratoria('SQBedjefe')

SQBescolari (escolari squared)

In [ ]:
exploratoria('SQBescolari')

SQBmeaned (square of the mean years of education of adults (>=18) in the household)

In [ ]:
exploratoria('SQBmeaned')

## ***SELEÇÃO DE VARIÁVEIS***

In [ ]:
treino = df_all[~df_all['Target'].isnull()]
valid = df_all[df_all['Target'].isnull()]

In [ ]:
'''
# Separar os dataframes
treino, teste = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

treino.shape, teste.shape
'''

In [ ]:
# Separando as colunas para treinamento
cols = [c for c in df_all.columns if c not in ['Target','Id','idhogar','dependency','edjefe','edjefa']]

In [ ]:
y_var = treino['Target']
x_var = treino[cols]
#x_var = train[['v2a1','hacdor','rooms']]

In [ ]:
reg = LassoCV()
reg.fit(x_var, y_var)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(x_var,y_var))
coef = pd.Series(reg.coef_, index = x_var.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
%%time
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Seleção de variáveis usando Regressão LASSO - Variáveis dummies")

In [ ]:
imp_coef[abs(imp_coef) != 0]

### Matriz de correlação

In [ ]:
variaveis = sorted(imp_coef[abs(imp_coef) != 0].index)

In [ ]:
#create a single data frame with both features and target by concatonating
boston_df=pd.concat([x_var[variaveis], y_var], axis=1)
boston_df.head()

#Visualize corelations
import seaborn as sns
plt.figure(figsize=(12,10))
ax=sns.heatmap(abs(boston_df.corr()), annot=True, cmap=sns.cubehelix_palette(20,  light=0.95, dark=0.15))
ax.xaxis.tick_top
plt.show()

## ***SPLIT DA BASE***

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x_var, y_var, test_size=0.20, random_state=27)

In [ ]:
x_treino.shape, x_teste.shape

## ***MODELAGEM***

In [ ]:
metricas = pd.DataFrame()
metricas['modelo'] = []
metricas['acuracia'] = []
metricas['ks'] = []

#### Base modelagem

In [ ]:
cols_mod = ['v2a1','agesq','SQBage','SQBedjefe','SQBescolari','SQBmeaned']

y_mod_treino = y_treino
x_mod_treino = x_treino[cols_mod]

y_mod_teste = y_teste
x_mod_teste = x_teste[cols_mod]

# Random Forest

In [ ]:
# Instanciar o modelo
rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

#### Treino

In [ ]:
# Treinando o modelo
rf.fit(x_mod_treino, y_mod_treino)
acuracia = rf.score(x_mod_treino, y_mod_treino)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### Teste

In [ ]:
acuracia = rf.score(x_mod_teste, y_mod_teste)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### KS

In [ ]:
y_logistic_train = rf.predict(x_mod_teste)
prob_logistic_train = rf.predict_proba(x_mod_teste)[:,1]

ks_table = pd.DataFrame()
ks_table['Target'] = y_logistic_train
ks_table['PROB_LOGISTIC'] = prob_logistic_train

ks_logistica_train = ks(data=ks_table,target="Target", prob="PROB_LOGISTIC")
ks_logistica_train = ks_logistica_train['KS'].max()

In [ ]:
d = {'modelo': ['Random Forest'], 'acuracia': [acuracia], 'ks': [ks_logistica_train]}
df = pd.DataFrame(data=d)
metricas = metricas.append(df)
metricas

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)

#### Treino

In [ ]:
gbm.fit(x_mod_treino, y_mod_treino)
acuracia = gbm.score(x_mod_treino, y_mod_treino)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### Teste

In [ ]:
acuracia = gbm.score(x_mod_teste, y_mod_teste)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### KS

In [ ]:
y_logistic_train = gbm.predict(x_mod_teste)
prob_logistic_train = gbm.predict_proba(x_mod_teste)[:,1]

ks_table = pd.DataFrame()
ks_table['Target'] = y_logistic_train
ks_table['PROB_LOGISTIC'] = prob_logistic_train

ks_logistica_train = ks(data=ks_table,target="Target", prob="PROB_LOGISTIC")
ks_logistica_train = ks_logistica_train['KS'].max()

In [ ]:
d = {'modelo': ['Gradient Boosting'], 'acuracia': [acuracia], 'ks': [ks_logistica_train]}
df = pd.DataFrame(data=d)
metricas = metricas.append(df)
metricas

# XGBoost

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)

#### Treino

In [ ]:
xgb.fit(x_mod_treino, y_mod_treino)
acuracia = gbm.score(x_mod_treino, y_mod_treino)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### Teste

In [ ]:
acuracia = xgb.score(x_mod_teste, y_mod_teste)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### KS

In [ ]:
y_logistic_train = xgb.predict(x_mod_teste)
prob_logistic_train = xgb.predict_proba(x_mod_teste)[:,1]

ks_table = pd.DataFrame()
ks_table['Target'] = y_logistic_train
ks_table['PROB_LOGISTIC'] = prob_logistic_train

ks_logistica_train = ks(data=ks_table,target="Target", prob="PROB_LOGISTIC")
ks_logistica_train = ks_logistica_train['KS'].max()

In [ ]:
d = {'modelo': ['XGBoost'], 'acuracia': [acuracia], 'ks': [ks_logistica_train]}
df = pd.DataFrame(data=d)
metricas = metricas.append(df)
metricas

# Adaboost

In [ ]:
# Trabalhando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)

#### Treino

In [ ]:
abc.fit(x_mod_treino, y_mod_treino)
acuracia = abc.score(x_mod_treino, y_mod_treino)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### Teste

In [ ]:
#gbm.fit(x_mod_teste, y_mod_teste)
acuracia = abc.score(x_mod_teste, y_mod_teste)
print('O modelo obteve %0.4f de acurácia.' % acuracia)

#### KS

In [ ]:
y_logistic_train = abc.predict(x_mod_teste)
prob_logistic_train = abc.predict_proba(x_mod_teste)[:,1]

ks_table = pd.DataFrame()
ks_table['Target'] = y_logistic_train
ks_table['PROB_LOGISTIC'] = prob_logistic_train

ks_logistica_train = ks(data=ks_table,target="Target", prob="PROB_LOGISTIC")
ks_logistica_train = ks_logistica_train['KS'].max()

In [ ]:
d = {'modelo': ['Adaboost'], 'acuracia': [acuracia], 'ks': [ks_logistica_train]}
df = pd.DataFrame(data=d)
metricas = metricas.append(df)
metricas

O XGBoost foi escolhido por ter o maior ks e a maior acurácia.

# Base de validação

In [ ]:
valid[cols_mod].isnull().sum()

In [ ]:
#%% faz predição com dataframe de teste
valid["Target"] = xgb.predict(valid[cols_mod]).astype(int)

valid["Target"].value_counts(normalize=True)

In [ ]:
valid[['Id', 'Target']].to_csv('submission.csv', index=False)